### 이미지파일이 저장된 경로를 설정해주세요
#### 바탕화면일 경우
* C:\\Users\컴퓨터이름\\Desktop\\breakout_bg.jpg

In [4]:
import SimpleGUICS2Pygame.simpleguics2pygame as simplegui 
import random
import math

#imaage load    
image = simplegui.load_image('C:\\Users\\dydeh\\breakout_bg.jpg')

# global constants
SIZE = (480,640)

LIVES = 3

TOP_BORDER = 60
BOTTOM_BORDER = 20

BLOCK_SIZE = (80,20)
BLOCK_INFO = (\
  {"color":"rgb(162, 71, 234)"	, "points":60},\
  #{"color":"rgb(162, 71, 234)"	, "points":60},\
  #{"color":"rgb(122, 157, 237)"	, "points":50},\
  #{"color":"rgb(122, 157, 237)"	, "points":50},\
  #{"color":"rgb(84, 237, 190)"	, "points":30},\
  #{"color":"rgb(84, 237, 190)"	, "points":30},\
  #{"color":"rgb(75, 219, 85)"	, "points":20},\
  #{"color":"rgb(75, 219, 85)"	, "points":20},\
  )

LEVEL_SPEED_FACTOR = 1.1
LEVEL_MAX_SPEED_FACTOR = 2.0

POINT_MULTIPLIER = 1.1

PADDLE_SIZE = (80,7)
PADDLE_MIN_POS = 0
PADDLE_MAX_POS = SIZE[0]-PADDLE_SIZE[0]
PADDLE_VEL = 8

SHOT_RADIUS = 5
SHOT_VEL = 4

# global variables
blocks = None
paddle_pos = None
paddle_vel = None
shot_pos = None
shot_vel = None
lives = None
level = None
points = None
show_help = None


class Ball():
    def __init__(self,shot_pos,radius,shot_vel):
        self.shot_pos = shot_pos
        self.radius = radius
        self.shot_vel = shot_vel
        
    def draw(self,canvas,shot_pos):
        canvas.draw_circle(shot_pos, self.radius, 1,"White", "White")
        shot_angle = math.pi/2 + random.choice((-1,1)) * (math.pi*0.01 + math.pi*0.25 * random.randrange(100)/100)
        sv = SHOT_VEL * min((LEVEL_SPEED_FACTOR**level), LEVEL_MAX_SPEED_FACTOR)
        shot_vel = [sv*math.cos(shot_angle), sv*math.sin(shot_angle)]


# 블록정보 생성
def block_info(pos, size, points, color):
    return {\
      "pos": tuple(pos),\
      "points": points,\
      "color": color,\
      "pos1": (pos[0]+1, pos[1]+size[1]/2),\
      "pos2": (pos[0]+size[0]-1, pos[1]+size[1]/2),\
      }


def launch_shot():
    global show_help, shot_pos, shot_vel
    show_help = False
    if lives==0:
        new_game()
    if len(blocks)==0:
        new_level()
    if shot_pos==None:
        shot_angle = math.pi/2 + random.choice((-1,1)) * (math.pi*0.01 + math.pi*0.25 * random.randrange(100)/100)
        shot_pos = [SIZE[0]/2, SIZE[1]/2]
        sv = SHOT_VEL * min((LEVEL_SPEED_FACTOR**level), LEVEL_MAX_SPEED_FACTOR)
        shot_vel = [sv*math.cos(shot_angle), sv*math.sin(shot_angle)]

# prepare for a new game
def new_game():
    global lives, level, points
    
    lives = LIVES
    level = 0
    points = 0

    new_level()
    
    
# start a new level
def new_level():
    global level, blocks, paddle_pos, paddle_vel, shot_pos, shot_vel
    
    level += 1
    
    blocks = []
    x0 = (SIZE[0] % BLOCK_SIZE[0])/2
    ncols = SIZE[0]//BLOCK_SIZE[0]
    for row in range(len(BLOCK_INFO)):
        for col in range(ncols):
            blocks.append(block_info(\
              (x0+col*BLOCK_SIZE[0],TOP_BORDER+row*BLOCK_SIZE[1]),\
              BLOCK_SIZE,\
              level*BLOCK_INFO[row]["points"],\
              BLOCK_INFO[row]["color"]\
              ))

    paddle_pos = [(PADDLE_MAX_POS+PADDLE_MIN_POS)/2, 
                  SIZE[1]-BOTTOM_BORDER]
    paddle_vel = 0
            
    shot_pos = None
    shot_vel = None

    
# draw handler	
def draw(canvas):
    global paddle_angle, shot_pos, lives, points, point_multiplier
    
    #배경 background
    canvas.draw_image(image, (300/2, 168/2), (300, 168), (0, 0), (480, 640))
    canvas.draw_image(image, (300/2, 168/2), (300, 168), (480, 0), (480, 640))
    canvas.draw_image(image, (300/2, 168/2), (300, 168), (0, 640), (480, 640))
    canvas.draw_image(image, (300/2, 168/2), (300, 168), (480, 640), (480, 640))
    
    
    canvas.draw_line([0,30],[SIZE[0], 30], 3, "White") 
    
    ball = Ball(shot_pos,5,4)  #Ball(shot_pos, radius, shot_vel)
    
    # update paddle position
    if paddle_vel!=0:
        paddle_pos[0] = max(PADDLE_MIN_POS, min(PADDLE_MAX_POS, paddle_pos[0] + paddle_vel))

    # check if shot exists
    if shot_pos!=None:
        # adjust shot for velocity 
        shot_pos[0] += shot_vel[0]
        shot_pos[1] += shot_vel[1]
        
        # check for shot hitting a block
        if shot_pos[1]<=TOP_BORDER+len(BLOCK_INFO)*BLOCK_SIZE[1]: # optimization
            block_to_remove = None
            sx = shot_pos[0]
            sy = shot_pos[1]
            sl = shot_pos[0]-ball.radius
            sr = shot_pos[0]+ball.radius
            st = shot_pos[1]-ball.radius
            sb = shot_pos[1]+ball.radius
            svx = shot_vel[0]
            svy = shot_vel[1]
            
            for b in blocks:
                bl = b["pos"][0]
                br = b["pos"][0]+BLOCK_SIZE[0]
                bt = b["pos"][1]
                bb = b["pos"][1]+BLOCK_SIZE[1]
                
                if (bt<st<bb and bl<sx<br and svy<0) or (bt<sb<bb and bl<sx<br and svy>0):
                    # bounce off bottom or top of block
                    shot_vel[1] *= -1
                    block_to_remove = b
                    break
                elif (bl<sr<br and bt<sy<bb and svx>0) or (bl<sl<br and bt<sy<bb and svx<0):
                    # bounce off left or right of block
                    shot_vel[0] *= -1
                    block_to_remove = b
                    break
            if block_to_remove!=None:
                blocks.remove(block_to_remove)
                points += int(point_multiplier*block_to_remove["points"])
                point_multiplier *= POINT_MULTIPLIER
                print(block_to_remove)
                
        # bounce shot off left/right/top walls
        if (shot_pos[0]-ball.radius<=0 and shot_vel[0]<0) or (shot_pos[0]+ball.radius>=SIZE[0] and shot_vel[0]>0):
            shot_vel[0] *= -1
        if shot_pos[1]-ball.radius-33<=0 and shot_vel[1]<0:
            shot_vel[1] *= -1
            
        # bounce off paddle
        if paddle_pos[0]<shot_pos[0]<paddle_pos[0]+PADDLE_SIZE[0] and paddle_pos[1]<shot_pos[1]+ball.radius<paddle_pos[1]+PADDLE_SIZE[1] and shot_vel[1]>0:
            relpos = (shot_pos[0]-paddle_pos[0]) /  PADDLE_SIZE[0]
            if relpos<0.3 or relpos>0.7:
                shot_vel[0] = -(shot_vel[0]+0.5)
            shot_vel[1] *= -1
            point_multiplier = '1
            print(point_multiplier)
            
        # check for shot passing paddle
        if shot_pos[1]+ball.radius>=SIZE[1]:
            shot_pos = None
            lives -= 1

        # check if level over
        if len(blocks)==0:
            shot_pos = None
            lives += 1
            
    # draw blocks
    for b in blocks:
        canvas.draw_line(b["pos1"], b["pos2"], BLOCK_SIZE[1]-2, b["color"])

    # draw paddle
    paddle_pos1 = (paddle_pos[0], paddle_pos[1]+PADDLE_SIZE[1]/2) #start point xy
    paddle_pos2 = (paddle_pos[0]+PADDLE_SIZE[0], paddle_pos[1]+PADDLE_SIZE[1]/2) #end point xy
    canvas.draw_line(paddle_pos1, paddle_pos2, PADDLE_SIZE[1], "white")

    # draw shot if needed
    if shot_pos!=None:
        canvas.draw_circle(shot_pos, ball.radius, 1, "white", "white")

    # draw text for score, level, and lives
    s = 18 #글자크기
    c = "white" #색상
    t = "Score: "+str(points) #점수
    p = [5,20] #위치
    canvas.draw_text(t, p, s, c) #text, pos, color, size
    
    t = "Level: "+str(level)
    p[0] = (SIZE[0]-frame.get_canvas_textwidth(t,s))/2
    canvas.draw_text(t, p, s+5, c)

    t = "Lives: "+str(lives)
    p[0] = SIZE[0]-frame.get_canvas_textwidth(t,s)-5
    canvas.draw_text(t, p, s, c)
    
    # 화면중간에 출력할 메시지
    if lives==0:
        canvas.draw_text("Game Over!", (SIZE[0]/2-100, (SIZE[1]/5)*3), 48, 'white', 'serif')
    elif shot_pos==None:
        canvas.draw_text("Press SPACEBAR to continue", (SIZE[0]/2-140, (SIZE[1]/5)*4), 24, 'white', 'serif')



def keydown(key):
    global paddle_vel
    if key==simplegui.KEY_MAP["left"]:
        paddle_vel = -6
    if key==simplegui.KEY_MAP["right"]:
        paddle_vel = 6
    if key==simplegui.KEY_MAP["space"]:
        launch_shot()
        
        
#keyup : 값을 입력중이 아닐 때 paddle속도는 0
def keyup(key):
    global paddle_vel
    if key==simplegui.KEY_MAP["left"]:
        paddle_vel = 0
    if key==simplegui.KEY_MAP["right"]:
        paddle_vel = 0
        
        
# initialize and run
frame = simplegui.create_frame("Breakout!", SIZE[0], SIZE[1])
new_game()
#show_help = True
frame.set_keydown_handler(keydown)
frame.set_keyup_handler(keyup)
frame.set_draw_handler(draw)
label = frame.add_label('SPACEBAR  to continue')
frame.start()

hit


ValueError: invalid literal for int() with base 10: 'hithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithithit'

In [42]:
cd

C:\Users\dydeh
